# Predicting phylum level taxonomic idendities of organisms based on codon usage levels

---

## Install some required packages used in this entire notebook

---
The following python packages are used in this notebook. Run the below cell to install them if you havent done so.


In [ ]:
!pip install wget requests selenium biopython pandas scikit-learn matplotlib

## Data download process


---


The data is available from https://hive.biochemistry.gwu.edu/dna.cgi?cmd=tissue_codon_usage&id=586358&mode=cocoputs. The Refseq_species.tsv contains all the codon frequencies of many species from NCBI. Firstly I tried to download via the download link with wget.

In [ ]:
import os
import wget
print(myfile)
data_folder = "./data"
filename = "codon_usage.tsv"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
filepath = os.path.join(data_folder,filename)
url = "https://hive.biochemistry.gwu.edu/dna.cgi?cmd=objFile&ids=586358&filename=Refseq_species.tsv&raw=1"
wget.download(url, filepath)
with open("data/codon_usage.tsv", "r") as f:
    print(f.readlines())

<Response [200]>
[]


However, the above approach did not work and returned an empty file. After examining the download process with the developers tool in Google Chrome, I realise it requires some headers for the GET request. I tried the below copying the headers in but it still did not work.

In [ ]:
import requests
headers = {
"Host": "hive.biochemistry.gwu.edu",
"Connection": "keep-alive",
"sec-ch-ua": '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
"sec-ch-ua-mobile": "?0",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "same-origin",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-User": "?1",
"Sec-Fetch-Dest": "document",
"Referer": "https://hive.biochemistry.gwu.edu/dna.cgi?cmd=tissue_codon_usage&id=586358&mode=cocoputs",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-US,en;q=0.9",

"Range": "bytes=783482-783482",
"If-Range": "1578517013:86024007"}
myfile = requests.get("https://hive.biochemistry.gwu.edu/dna.cgi?cmd=objFile&ids=586358&filename=Refseq_species.tsv&raw=1", headers=headers)
print(myfile.content)

b''


Once again, it is empty. Hence, I resorted to using Selenium and navigating the browser. First, I downloaded the Chrome driver. Then, I used the browser developer tools to examine the webpage and copy the XPATH of the elements that have to be clicked. On normal navigation, it requires 2 clicks to initate the download of the file. 

In [ ]:
from zipfile import ZipFile
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

#downloading and extracting the chrome drive
wget.download("https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip")
file_name = "chromedriver_win32.zip"
with ZipFile(file_name, 'r') as zip:
    zip.extractall()

driver = webdriver.Chrome(executable_path=r'./chromedriver.exe')
driver.get("https://hive.biochemistry.gwu.edu/dna.cgi?cmd=tissue_codon_usage&id=586358&mode=cocoputs")
timeout = 60
try:
    element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="results"]/li[3]/a'))
    WebDriverWait(driver, timeout).until(element_present)
    download_tab = driver.find_element_by_xpath("/html/body/form[2]/div/div/div[4]/div[1]/div[1]/div[2]/ul/li[3]/ul/li[3]/a")
    download_tab.click()
    print("Clicked the first button!")
except TimeoutException:
    print ("Timed out waiting for page to load")

try:
    element_present = EC.presence_of_element_located((By.XPATH, '/html/body/form[2]/div/div/div[7]/div[1]/div[2]/div/div/div[2]/table/tbody/tr[18]/td[2]'))
    WebDriverWait(driver, timeout).until(element_present)
    download = driver.find_element_by_xpath('/html/body/form[2]/div/div/div[7]/div[1]/div[2]/div/div/div[2]/table/tbody/tr[18]/td[2]')
    download.click()
    print("Clicked the second tab!")
except TimeoutException:
    print ("Timed out waiting for page to load")

driver.close()

Clicked the first button!
Timed out waiting for page to load


However, this method ultimately did not work as the second table did not appear after clicking the first click. I was unable to make more progress due to my limited knowledge of HTML and JavaScript. Ultimately, I postulates that there are safeguards put in place to limit any automated downloading/conceal the download as they are an academic institute and may be afraid of request overload. I moved on by downloading the data manually from https://hive.biochemistry.gwu.edu/dna.cgi?cmd=tissue_codon_usage&id=586358&mode=cocoputs (as of 13th of March). 

## Data exploration

---


Here, we will look at how many entries are there in the file. 

In [ ]:
import pandas as pd
#read in CoCoPUTs file
df = pd.read_table("data/o586358-Refseq_species.tsv",index_col=False,low_memory=False)
print(df.shape)
df.head()

(191025, 76)


,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG
0,refseq,GCF_002287175.1,2161,Methanobacterium bryantii,genomic,11,3266,910953,35.52,45.52,...,2404,1515,12496,7687,14043,8418,18911,8417,26631,8789
1,refseq,GCF_000302455.1,1204725,Methanobacterium formicicum DSM 3637,genomic,11,2520,719599,40.88,49.69,...,2968,3068,10562,4882,8084,6214,17552,5296,21059,7069
2,refseq,GCF_000762265.1,2162,Methanobacterium formicicum,genomic,11,2376,678104,43.72,51.53,...,3067,5204,9335,4793,6499,5439,15945,5847,19130,7839
3,refseq,GCF_001458655.1,2162,Methanobacterium formicicum,genomic,11,2372,690178,43.50,51.30,...,3102,5182,9545,4797,6613,5472,16184,5887,19385,7934
4,refseq,GCF_000016525.1,420247,Methanobrevibacter smithii ATCC 35061,genomic,11,1757,557617,32.06,43.34,...,405,168,8234,3248,10191,2880,14664,2965,16981,2107


In [ ]:
print(list(df))
print(df.iloc[:, 0:6].nunique())
df.iloc[:,4].unique()

['Division', 'Assembly', 'Taxid', 'Species', 'Organelle', 'Translation Table', '# CDS', '# Codons', 'GC%', 'GC1%', 'GC2%', 'GC3%', 'TTT', 'TTC', 'TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG', 'ATT', 'ATC', 'ATA', 'ATG', 'GTT', 'GTC', 'GTA', 'GTG', 'TAT', 'TAC', 'TAA', 'TAG', 'CAT', 'CAC', 'CAA', 'CAG', 'AAT', 'AAC', 'AAA', 'AAG', 'GAT', 'GAC', 'GAA', 'GAG', 'TCT', 'TCC', 'TCA', 'TCG', 'CCT', 'CCC', 'CCA', 'CCG', 'ACT', 'ACC', 'ACA', 'ACG', 'GCT', 'GCC', 'GCA', 'GCG', 'TGT', 'TGC', 'TGA', 'TGG', 'CGT', 'CGC', 'CGA', 'CGG', 'AGT', 'AGC', 'AGA', 'AGG', 'GGT', 'GGC', 'GGA', 'GGG']
Division                  1
Assembly             177439
Taxid                 59670
Species               59060
Organelle                 4
Translation Table        11
dtype: int64


array(['genomic', 'mitochondrion', 'chloroplast', 'plastid'], dtype=object)

As we can see here, we have 191025 samples with 76 variables. A quick glance at the number of unique values show that the data requires cleaning. Firstly, we have less unique assemblies than number of rows, meaning that some of the assemblies are repeated or NaN. Even though the number of species and taxid differs, here it is acceptable as there might taxonomic IDs only goes up till species levels but there might be subspecies especially for microbes. We also have different type of DNA origins, here we focus on genomic DNA as those genes in mitochondrion, plastid or chloroplast might be shared between distantly related species and skew the differences. 

In [ ]:
#show duplicated entries
df[df.duplicated(['Assembly'])]

,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG
658,refseq,GCF_000513435.1,1173487,Halorubrum sp. AJ67,genomic,11,3078,763055,64.68,67.31,...,8366,17661,3223,10213,1988,2032,7707,29947,8715,16798
660,refseq,GCF_000513435.1,1173487,Halorubrum sp. AJ67,genomic,11,3083,764041,64.69,67.32,...,8369,17683,3223,10222,1988,2032,7711,30002,8725,16820
719,refseq,NaN,1407055,Candidatus Nitrosotenuis uzonensis,genomic,11,1,200,55.83,60.50,...,1,0,1,1,2,2,0,2,5,0
720,refseq,GCF_000723185.1,1407055,Candidatus Nitrosotenuis uzonensis,genomic,11,1876,493394,42.86,49.86,...,2243,824,3560,5730,9304,5257,6294,10132,12660,3397
4956,refseq,NaN,1354719,Helicobacter pylori HP87P7,genomic,11,4,1064,37.53,40.04,...,11,8,10,22,21,21,9,10,14,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191020,refseq,NaN,1923763,Xingshan nematode virus 4,genomic,0,5,3630,38.21,43.91,...,49,13,60,29,72,21,40,19,85,41
191021,refseq,NaN,1923772,Xinzhou nematode virus 4,genomic,0,5,3652,39.91,45.29,...,39,11,71,30,73,12,38,21,99,34
191022,refseq,NaN,1415147,Xylella phage Salvo,genomic,11,72,17848,63.15,62.21,...,58,140,30,344,10,22,163,993,71,109
191023,refseq,NaN,1415148,Xylella phage Sano,genomic,11,77,17851,62.46,61.59,...,71,137,41,340,10,24,185,993,74,95


Interestingly, there are multiple entries with the same assembly ID but differing number of coding sequences (# CDS). This may be due to the different gene open reading fram predictions from different softwares. To simplify the data, we shall only keep the assembly with the largest number of CDS. Furhtermore, we shall remove those without assembly IDs, as they are just draft collections of contigs without proper curated annotations. We also remove those with low number of codons.

In [ ]:
df = df[df["Organelle"] == "genomic"] #keep genomic
df = df[df["Assembly"].notna()] #drop NaN
df = df[df["# Codons"] > 20000] #remove low codon counts
df = df.sort_values("# CDS", ascending=False).drop_duplicates("Assembly").sort_index()
print(df.shape)
df[df.duplicated(["Assembly"])]

(177437, 76)


,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG


Looks like the data is cleaned! We shall check for any NAs just to be safe..

In [ ]:
print(df.isnull().any().any())
df.head()

False


,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG
0,refseq,GCF_002287175.1,2161,Methanobacterium bryantii,genomic,11,3266,910953,35.52,45.52,...,2404,1515,12496,7687,14043,8418,18911,8417,26631,8789
1,refseq,GCF_000302455.1,1204725,Methanobacterium formicicum DSM 3637,genomic,11,2520,719599,40.88,49.69,...,2968,3068,10562,4882,8084,6214,17552,5296,21059,7069
2,refseq,GCF_000762265.1,2162,Methanobacterium formicicum,genomic,11,2376,678104,43.72,51.53,...,3067,5204,9335,4793,6499,5439,15945,5847,19130,7839
3,refseq,GCF_001458655.1,2162,Methanobacterium formicicum,genomic,11,2372,690178,43.50,51.30,...,3102,5182,9545,4797,6613,5472,16184,5887,19385,7934
4,refseq,GCF_000016525.1,420247,Methanobrevibacter smithii ATCC 35061,genomic,11,1757,557617,32.06,43.34,...,405,168,8234,3248,10191,2880,14664,2965,16981,2107


## Preparing data for training and testing

---


We will split the data up, using the codons as variables (total of 64) and using the taxonomy IDs as class labels. We will need to parse the taxonomy tree for the lineage of each taxonomy id. 

In [ ]:
from Bio import Entrez
#entrez records has maximum length of 10000
total_species = len(df.iloc[:, 2].unique())
i=0
taxid = []
while i < total_species:
    if i == 50000:
        i_end = total_species
    else:
        i_end = i+10000
    handle = Entrez.efetch(db="Taxonomy", id=df.iloc[:, 2].unique()[i:i_end], retmode="xml")
    records = Entrez.read(handle)
    taxid.append(records)
    i+=10000

C:\Users\YZ\miniconda3\envs\project\lib\site-packages\Bio\Entrez\__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


In [ ]:
#create empty taxa table to keep track of each taxa lineage
#0 is a safe value as 0 correspond to the root node of the taxanomy tree at NCBI, which is a non valid annotation
tax_df = pd.DataFrame(0, index=range(total_species), columns=range(7))
major_taxonomy_levels = ["superkingdom", "phylum", "class", "order", "family", "genus", "Taxid"]
tax_df.columns = major_taxonomy_levels

In [ ]:
#populate the taxa table for each record
i=0
for records in taxid:
    for record in records:
        tax_df.iloc[i,6] = int(record["TaxId"])
        for tax in record["LineageEx"]:
            if tax["Rank"] == "superkingdom":
                tax_df.iloc[i,0] = int(tax["TaxId"])
            elif tax["Rank"] == "phylum":
                tax_df.iloc[i,1] = int(tax["TaxId"])
            elif tax["Rank"] == "class":
                tax_df.iloc[i,2] = int(tax["TaxId"])
            elif tax["Rank"] == "order":
                tax_df.iloc[i,3] = int(tax["TaxId"])
            elif tax["Rank"] == "family":
                tax_df.iloc[i,4] = int(tax["TaxId"])
            elif tax["Rank"] == "genus":
                tax_df.iloc[i,5] = int(tax["TaxId"])
        i+=1
tax_df       

,superkingdom,phylum,class,order,family,genus,Taxid
0,2157,28890,183925,2158,2159,2160,2161
1,2157,28890,183925,2158,2159,2160,1204725
2,2157,28890,183925,2158,2159,2160,2162
3,2157,28890,183925,2158,2159,2172,420247
4,2157,28890,183925,2158,2159,2172,521002
...,...,...,...,...,...,...,...
50392,2759,7711,8292,8445,1277737,264009,1415580
50393,2759,7711,186623,7952,7953,75365,1608454
50394,2759,7711,186623,41712,31092,91732,1676925
50395,2759,7711,1338369,8459,8487,38771,1825980


In [ ]:
print((tax_df == 0).sum(axis=0)) #find number of zeroes in each taxa
print(len(tax_df.iloc[:, 1].unique())) #find number of unique taxa at phylum level
from collections import Counter
Counter(tax_df["phylum"]).values() #check distribution of number records for each phyla

superkingdom       0
phylum            71
class            785
order            424
family           837
genus           1077
Taxid              0
dtype: int64
88


dict_values([603, 91, 23, 33, 3, 1, 1, 71, 1, 21866, 8127, 4, 2663, 148, 615, 23, 13256, 116, 74, 183, 134, 43, 11, 32, 26, 547, 354, 83, 9, 132, 29, 50, 3, 1, 1, 5, 4, 3, 1, 15, 2, 6, 2, 2, 22, 1, 5, 1, 3, 1, 4, 1, 1, 2, 1, 3, 1, 1, 6, 4, 210, 4, 52, 8, 3, 9, 4, 4, 1, 9, 143, 1, 2, 309, 1, 1, 1, 1, 108, 8, 1, 2, 1, 7, 13, 4, 35, 1])

There appears to be some data with missing taxonomic annotations. Furthermore, some phyla has many examples while others only have a handful. We will only keep those with > 100 examples.

In [ ]:
#keep taxa with only > 100 samples
phylum_to_keep = [key for key,value in Counter(tax_df["phylum"]).items() if value > 100]
tax_df = tax_df.loc[tax_df["phylum"].isin(phylum_to_keep)]
taxid_to_keep = tax_df["Taxid"]
df = df.loc[df["Taxid"].isin(taxid_to_keep)]
df

,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG
0,refseq,GCF_002287175.1,2161,Methanobacterium bryantii,genomic,11,3266,910953,35.52,45.52,...,2404,1515,12496,7687,14043,8418,18911,8417,26631,8789
1,refseq,GCF_000302455.1,1204725,Methanobacterium formicicum DSM 3637,genomic,11,2520,719599,40.88,49.69,...,2968,3068,10562,4882,8084,6214,17552,5296,21059,7069
2,refseq,GCF_000762265.1,2162,Methanobacterium formicicum,genomic,11,2376,678104,43.72,51.53,...,3067,5204,9335,4793,6499,5439,15945,5847,19130,7839
3,refseq,GCF_001458655.1,2162,Methanobacterium formicicum,genomic,11,2372,690178,43.50,51.30,...,3102,5182,9545,4797,6613,5472,16184,5887,19385,7934
4,refseq,GCF_000016525.1,420247,Methanobrevibacter smithii ATCC 35061,genomic,11,1757,557617,32.06,43.34,...,405,168,8234,3248,10191,2880,14664,2965,16981,2107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181848,refseq,GCF_901765095.1,1415580,Microcaecilia unicolor,genomic,0,35405,20605114,46.78,52.18,...,137037,136305,330959,359823,321754,242704,264062,274277,410074,241232
181850,refseq,GCF_001515605.1,1608454,Sinocyclocheilus anshuiensis,genomic,0,67753,41525357,50.84,54.62,...,266568,284411,622434,819038,615112,470561,550460,697798,812809,451550
181852,refseq,GCF_002872115.1,1676925,Paramormyrops kingsleyae,genomic,0,55174,37697302,56.10,57.17,...,189905,463292,403279,918378,335427,566434,388203,913712,520100,646555
181853,refseq,GCF_007399415.2,1825980,Gopherus evgoodei,genomic,0,49709,34253340,49.30,54.06,...,199608,300008,500034,677088,516372,422769,365846,548610,609307,517676


In [ ]:
#change to percentage
df.iloc[:,12:76] = df.iloc[:,12:76].div(df["# Codons"].values, axis=0)
#add phylum data
df = df.merge(tax_df[["phylum", "Taxid"]], how="left")

We shall save the data for future convenience.

In [ ]:
#split into X and y data
X = df.iloc[:, 12:76]
y = df.iloc[:, 76]
#save data
X.to_csv("data/X_cleaned.csv")
y.to_csv("data/y_cleaned.csv")

Side note on not normalizing and scaling the data: 

---


Due to the fact that the genome sizes and hence the number of coding sequences vary wildly from single cell organisms such as bacteria to multicellular organisms such as mammmals, I have decided to just use the percentage of each codon the input variables for the machine learning algorithms. However, this leads to a whole class of problems on its own, a field termed as compositional data analysis (CoDA). In brief, the constant sum restraint means that each variable is negatively correlated to every other variable. Furthermore, spurious correlations occurs due to the shared denominator between all variables. In statistical analysis, these spurious correlations lead to high number of false positives. In machine learning, it means that some algorithms are not suitable for the task at hand, such as Naive Bayes, which was initially used in this project. One possible solution is to transfrom the data with the class of transformation called log ratio transformation. However, how to best prepare the data and apply these transformations is an active area of research and hence I decided not to proceed with that here. 

## Training machine learning algorithms

---

In this section, we will explore 3 different machine learning algorithms for classification. They are the k-Nearest Neighbours, Random Forest, and XGBoost. All methods will be evaluated with the `classification_report` from `sklearn.metrics`, which shows the weighted F1 scores, precision and recall among others. The F1 score, which is the harmonic mean of precision and recall, is especially important in this case with imbalanced data classes. 

First, we will load in the data cleaned from before.

In [ ]:
#local machine ver
import pandas as pd
import numpy as np
X = pd.read_csv("data/X_cleaned.csv", index_col=0)
y = pd.read_csv("data/y_cleaned.csv", index_col=0)

In [19]:
#colab ver
from google.colab import drive
import pandas as pd

drive.mount("/content/gdrive")
X = pd.read_csv("/content/gdrive/MyDrive/ce9010/X_cleaned.csv", index_col=0) #replace with path to your file
y = pd.read_csv("/content/gdrive/MyDrive/ce9010/y_cleaned.csv", index_col=0) #replace with path to your file

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Training k-Nearest Neighbour classifier
---
k-Nearest Neighbour is a simple algorithm that works on the principle that similar objects have similar characteristics, hence, they lie close to each other in the variable space. The class for an unknown object is assigned based on the most common class of k nearest objects. Here, we will use the default  Euclidean distance metric and vary the k variable for optimal predictive power. We will use the `StratifiedKFold` from `sklearn.model_selection` to find the best k value. 



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report

# rename the classes
le = preprocessing.LabelEncoder()
y_encoded = le.fit_transform(y[:,0])

# knn k choosing
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded,stratify=y_encoded,test_size=0.3,random_state=42) #70% training and 30% test
skf = StratifiedKFold(n_splits=10)
k = 1
for train_index, test_index in skf.split(X_train, y_train):
    kfold_X_train = X_train[train_index,:]
    kfold_y_train = y_train[train_index]
    kfold_X_test = X_train[test_index,:]
    kfold_y_test = y_train[test_index]
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(kfold_X_train,kfold_y_train)
    preds = knn.predict(kfold_X_test)
    print(f"KNN report for {k} Nearest Neighbours:")
    print(classification_report(kfold_y_test, preds,digits=4))
    k+=1

KNN report for 1 Nearest Neighbours:
              precision    recall  f1-score   support

           0     1.0000    0.9905    0.9952       317
           1     0.9756    1.0000    0.9877        40
           2     0.9987    0.9994    0.9991      7025
           3     0.9991    0.9993    0.9992      4573
           4     1.0000    1.0000    1.0000        12
           5     1.0000    0.9333    0.9655        15
           6     1.0000    1.0000    1.0000        10
           7     1.0000    1.0000    1.0000        22
           8     1.0000    0.9808    0.9903        52
           9     1.0000    1.0000    1.0000        18
          10     0.8750    1.0000    0.9333         7
          11     1.0000    0.8947    0.9444        19
          12     0.9984    1.0000    0.9992      1266
          13     0.9167    0.9167    0.9167        12
          14     1.0000    0.9595    0.9793        74
          15     1.0000    1.0000    1.0000        28
          16     1.0000    1.0000    1.0000 

In general, the algorithm actually perform best at k = 1. As the number of neighbour to take into consideration increases, the performance of the rare classes decreases, especially class 13. Recall for class 13 fluctuates wildly, meaning that the samples from this class are classified wrongly (high number of false negatives). 

In [ ]:
#get actual phylum for class 13
print(le.inverse_transform([13]))
print(sum(y_encoded[y_encoded == 13]))

[203682]
2275


The taxonomy ID belongs to Planctomycetes, with 2275 samples in our data. Planctomycetes have been known to occupy diverse niches, and are very metabolically flexible. Their flexibilty may lead to them occupying a dispersed area of the variable space, such that low number of nearest neighbours are able to correct assign the class, while when taking into account more neighbours encroach into the variable subspace occupied by other classes.

In [ ]:
# knn on test set
k = 1
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train,y_train) #70% training and 30% test
preds = knn.predict(X_test)
knn_report = classification_report(y_test,preds,digits=4, output_dict=True)
print(f"KNN report for {k} Nearest Neighbours:")
print(classification_report(y_test, preds,digits=4))

KNN report for 1 Nearest Neighbours:
              precision    recall  f1-score   support

           0     0.9949    0.9963    0.9956      1359
           1     0.9882    0.9882    0.9882       170
           2     0.9990    0.9993    0.9992     30104
           3     0.9990    0.9992    0.9991     19598
           4     0.9800    0.9608    0.9703        51
           5     1.0000    0.9524    0.9756        63
           6     1.0000    0.9767    0.9882        43
           7     0.9894    1.0000    0.9947        93
           8     1.0000    0.9776    0.9887       223
           9     1.0000    0.9870    0.9935        77
          10     0.9697    1.0000    0.9846        32
          11     0.9737    0.9136    0.9427        81
          12     0.9982    0.9978    0.9980      5430
          13     0.9434    0.9615    0.9524        52
          14     0.9874    0.9874    0.9874       317
          15     0.9831    0.9831    0.9831       118
          16     1.0000    0.9852    0.9926 

**Expected output:**

```
KNN report for 1 Nearest Neighbours:
              precision    recall  f1-score   support

           0     0.9949    0.9963    0.9956      1359
           1     0.9882    0.9882    0.9882       170
           2     0.9990    0.9993    0.9992     30104
           3     0.9990    0.9992    0.9991     19598
           4     0.9800    0.9608    0.9703        51
           5     1.0000    0.9524    0.9756        63
           6     1.0000    0.9767    0.9882        43
           7     0.9894    1.0000    0.9947        93
           8     1.0000    0.9776    0.9887       223
           9     1.0000    0.9870    0.9935        77
          10     0.9697    1.0000    0.9846        32
          11     0.9737    0.9136    0.9427        81
          12     0.9982    0.9978    0.9980      5430
          13     0.9434    0.9615    0.9524        52
          14     0.9874    0.9874    0.9874       317
          15     0.9831    0.9831    0.9831       118
          16     1.0000    0.9852    0.9926       203

    accuracy                         0.9986     58014
   macro avg     0.9886    0.9804    0.9843     58014
weighted avg     0.9986    0.9986    0.9985     58014
```

### Training Random Forest classifier


---

Random forest is an ensemble learning method that operates on the consensus of multiple decision trees. In random forest method, there are many hyperparameters to tune for optimal performance. We will use `GridSearchCV` from `sklearn.model_selection` in order to select our best parameters. As hyperparameter tuning takes a lot of time especially with such a big data set, the blow code block is only a representative example of an optimising run. The full list of hyperparameters searched are as given below:

```python
param_grid = {
    'bootstrap': [True],
    'max_depth': [5,10,,20,25,30,35],
    'max_features': [10,20,25,30,32,35,38,40,45],
    'min_samples_leaf': [1,3,5,10],
    'min_samples_split': [1,3,5,10],
    'n_estimators': [25,50,75,100,125]
}
```

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(random_state = 42)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded,stratify=y_encoded,test_size=0.3,random_state=42) # 70% training and 30% test

param_grid = {
    'bootstrap': [True],
    'max_depth': [20,30],
    'max_features': [35],
    'min_samples_leaf': [5],
    'min_samples_split': [5],
    'n_estimators': [100]
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 45.1min finished


{'bootstrap': True,
 'max_depth': 30,
 'max_features': 35,
 'min_samples_leaf': 5,
 'min_samples_split': 5,
 'n_estimators': 100}

**Expected output**:
```
Fitting 3 folds for each of 2 candidates, totalling 6 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 65.9min finished
{'bootstrap': True,
 'max_depth': 30,
 'max_features': 35,
 'min_samples_leaf': 5,
 'min_samples_split': 5,
 'n_estimators': 100}
 ```

In [ ]:
# Using the best params from GridSearchCV
rf = RandomForestClassifier(n_estimators=100, max_depth=30, max_features=35,
                            min_samples_leaf=5,min_samples_split=5,random_state = 42)
rf.fit(X_train,y_train)
preds = rf.predict(X_test)
rf_report = classification_report(y_test,preds,digits=4, output_dict=True)
print(classification_report(y_test,preds,digits=4))


              precision    recall  f1-score   support

           0     0.9926    0.9875    0.9900      1359
           1     0.9880    0.9647    0.9762       170
           2     0.9971    0.9992    0.9981     30104
           3     0.9967    0.9985    0.9976     19598
           4     1.0000    0.9608    0.9800        51
           5     0.9828    0.9048    0.9421        63
           6     1.0000    0.9535    0.9762        43
           7     1.0000    1.0000    1.0000        93
           8     0.9950    0.9013    0.9459       223
           9     0.9737    0.9610    0.9673        77
          10     1.0000    1.0000    1.0000        32
          11     1.0000    0.7531    0.8592        81
          12     0.9948    0.9954    0.9951      5430
          13     0.9189    0.6538    0.7640        52
          14     1.0000    0.9558    0.9774       317
          15     0.9646    0.9237    0.9437       118
          16     1.0000    0.9704    0.9850       203

    accuracy              

### Training XGBoost classifier


---


XGBoost is another ensemble method, where gradient boosting is incorporated to improve performance. This is done via gradient descent hence the name. Additionally, XGBoost gives the option of performing regularisation, preventing overfitting. We will utilise the hyperparameters fround from Random Forest and see if there is improvement with XGBoost. 

In [23]:
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# rename the classes
le = preprocessing.LabelEncoder()
y_encoded = le.fit_transform(y[:,0])

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded,stratify=y_encoded,test_size=0.2,random_state=42) # 80% training and 20% test
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train,stratify=y_train,test_size=0.25,random_state=42) # 60% training and 20% eval
dtrain = xgb.DMatrix(X_train, label=y_train)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)

param = {"objective":"multi:softmax", "n_estimators":100, "max_depth":30, 
         "min_child_weight":5,"lambda":1.5, "alpha":1," colsample_bytree":35/64, 
         "learning_rate":0.1, "num_class":17}
evallist = [(dtrain, 'train'),(deval, 'eval')]
xg_reg = xgb.train(param,dtrain,num_boost_round=1000,evals=evallist,early_stopping_rounds=10)

[0]	train-merror:0.01073	eval-merror:0.014247
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.007507	eval-merror:0.010006
[2]	train-merror:0.006283	eval-merror:0.008558
[3]	train-merror:0.005757	eval-merror:0.007886
[4]	train-merror:0.005232	eval-merror:0.007291
[5]	train-merror:0.004792	eval-merror:0.006878
[6]	train-merror:0.004318	eval-merror:0.006464
[7]	train-merror:0.004163	eval-merror:0.006283
[8]	train-merror:0.003887	eval-merror:0.006024
[9]	train-merror:0.003723	eval-merror:0.005895
[10]	train-merror:0.003577	eval-merror:0.005637
[11]	train-merror:0.003448	eval-merror:0.00543
[12]	train-merror:0.003284	eval-merror:0.005197
[13]	train-merror:0.003172	eval-merror:0.005016
[14]	train-merror:0.002999	eval-merror:0.004964
[15]	train-merror:0.002905	eval-merror:0.004861
[16]	train-merror:0.002784	eval-merror:0.004654
[17]	train-merror:0.002689	eval-merror:0.004525
[

AttributeError: ignored

In [30]:
preds = xg_reg.predict(dtest)

# evaluate predictions
xgb_report = classification_report(y_test,preds,digits=4, output_dict=True)
print(classification_report(y_test,preds,digits=4))

              precision    recall  f1-score   support

           0     0.9978    0.9948    0.9963      1359
           1     1.0000    0.9706    0.9851       170
           2     0.9987    0.9997    0.9992     30104
           3     0.9988    0.9993    0.9990     19598
           4     1.0000    1.0000    1.0000        51
           5     0.9836    0.9524    0.9677        63
           6     1.0000    0.9302    0.9639        43
           7     1.0000    1.0000    1.0000        93
           8     1.0000    0.9731    0.9864       223
           9     1.0000    0.9870    0.9935        77
          10     1.0000    1.0000    1.0000        32
          11     1.0000    0.9630    0.9811        81
          12     0.9989    0.9972    0.9981      5430
          13     0.9583    0.8846    0.9200        52
          14     0.9843    0.9905    0.9874       317
          15     0.9915    0.9915    0.9915       118
          16     1.0000    0.9803    0.9900       203

    accuracy              